# How to deal with timestamps

## Install

In [2]:
SYFT_VERSION = ">=0.8.1b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
!pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [3]:
import syft as sy
sy.requires(SYFT_VERSION)

✅ The installed version of syft==0.8.1b2 matches the requirement >=0.8.1b0 and the requirement <0.9


In [4]:
node = sy.orchestra.launch(name="pandas-test-domain-1", port=8080, reset=True)

Starting pandas-test-domain-1 server on 0.0.0.0:8080
SQLite Store Path:
!open file:///var/folders/6_/7xx0tpq16h9cn40mq4w5gjk80000gn/T/eee436ccb7f044c4815c373f3fcf283d.sqlite



# Data owner: upload dataset

In [5]:
root_domain_client = node.login(email="info@openmined.org", password="changethis")

## Load data

It's not obvious how to deal with Unix timestamps in pandas -- it took me quite a while to figure this out. The file we're using here is a popularity-contest file I found on my system at /var/log/popularity-contest.

In [6]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from syft.service.project.project import Project
from syft.util.util import autocache, PANDAS_DATA

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 3)
plt.rcParams['font.family'] = 'sans-serif'

In [7]:
popcon = pd.read_csv(autocache(f"{PANDAS_DATA}/popularity-contest"), sep=' ', )[:-1]
popcon.columns = ['atime', 'ctime', 'package-name', 'mru-program', 'tag']

In [8]:
set(popcon["tag"])

{'<OLD>', '<RECENT-CTIME>', nan}

## Create mock data

Lets create a mock dataset

In [9]:
# weather_types = set(weather_2012_final.Weather.to_list())

In [10]:
from random import randint
import random

In [11]:
def get_random_timestamp():
    return "135" + "".join([str(randint(0,9)) for i in range(6)])

In [12]:
def get_mock_date(i):
    return str(parse('Jun 1 2010') + timedelta(days=i))

def get_mock_row(i):
    res = dict()
    for k, function in mock_functions.items():
        res[k] = function()
    return res
    

In [13]:
mock_functions = {'atime': lambda: get_random_timestamp(),
                  'ctime': lambda: get_random_timestamp(),
                  'package-name': lambda: random.choice([
                      'libghc-stm-dev',
                      'libqtdee-dev',
                      'geoclue-ubuntu-geoip',
                      'libdesktop-agnostic0',
                      'ubuntu-extras-keyring',
                      'libbsd0',
                      'libxres-dev']),
                  'mru-program': lambda: random.choice([
                      '/usr/bin/opam',
                      '/usr/bin/onboard',
                      '/lib/init/upstart-job',
                      '/usr/bin/page',
                  ]),
                  'tag': lambda: random.choice(['<OLD>', '<RECENT-CTIME>', 'nan'])}


In [14]:
mock = pd.DataFrame(data=[get_mock_row(i) for i in range(len(popcon))],
                    columns=popcon.columns)
    


Upload the data

In [15]:
dataset = sy.Dataset(name="test", asset_list=[sy.Asset(name=f"weather", data=popcon,
                                                       mock=mock, mock_is_real=False)])
root_domain_client.upload_dataset(dataset)


100%|█| 1/1 [00:00<00

Uploading: weather


<class 'syft.service.response.SyftSuccess'>: Dataset Added

In [16]:
popcon.head()

,atime,ctime,package-name,mru-program,tag
0,1387295797,1367633260,perl-base,/usr/bin/perl,NaN
1,1387295796,1354370480,login,/bin/su,NaN
2,1387295743,1354341275,libtalloc2,/usr/lib/x86_64-linux-gnu/libtalloc.so.2.0.7,NaN
3,1387295743,1387224204,libwbclient0,/usr/lib/x86_64-linux-gnu/libwbclient.so.0,<RECENT-CTIME>
4,1387295742,1354341253,libselinux1,/lib/x86_64-linux-gnu/libselinux.so.1,NaN


## Create user account

In [17]:
user = root_domain_client.register(name="Jane Doe", email="jane@caltech.edu",
                            password="abc123", institution="Caltech", website="https://www.caltech.edu/")
# todo: give user data scientist role
guest_domain_client = node.client
guest_client = guest_domain_client.login(email="jane@caltech.edu", password="abc123")

# Data scientist: create syft_function

In [18]:
import numpy as np
import pandas as pd

## Summary

It's not obvious how to deal with Unix timestamps in pandas -- it took me quite a while to figure this out. The file we're using here is a popularity-contest file I found on my system at `/var/log/popularity-contest`.

## Get mocks

In [19]:
ds = guest_domain_client.datasets[0]

In [20]:
asset = ds.assets[0]

In [21]:
popcon = asset.mock.syft_action_data

## Parsing Unix timestamps

The colums are the access time, created time, package name, recently used program, and a tag



In [22]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,135586904,135870262,libxres-dev,/usr/bin/opam,<RECENT-CTIME>
1,135474582,135178396,libdesktop-agnostic0,/usr/bin/onboard,<OLD>
2,135917928,135246781,libdesktop-agnostic0,/usr/bin/opam,<OLD>
3,135551200,135543162,geoclue-ubuntu-geoip,/lib/init/upstart-job,nan
4,135132986,135276441,libqtdee-dev,/usr/bin/page,<RECENT-CTIME>


The magical part about parsing timestamps in pandas is that numpy datetimes are already stored as Unix timestamps. So all we need to do is tell pandas that these integers are actually datetimes -- it doesn't need to do any conversion at all.

We need to convert these to ints to start:

In [23]:
popcon['atime'] = popcon['atime'].astype(int)
popcon['ctime'] = popcon['ctime'].astype(int)

Every numpy array and pandas series has a dtype -- this is usually `int64`, `float64`, or `object`. Some of the time types available are `datetime64[s]`, `datetime64[ms]`, and `datetime64[us]`. There are also `timedelta` types, similarly.

We can use the `pd.to_datetime` function to convert our integer timestamps into datetimes. This is a constant-time operation -- we're not actually changing any of the data, just how pandas thinks about it.

In [24]:
popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')

If we look at the dtype now, it's `<M8[ns]`. As far as I can tell `M8` is secret code for `datetime64`.

In [25]:
popcon['atime'].dtype

dtype('<M8[ns]')

So now we can look at our `atime` and `ctime` as dates!

In [26]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,1974-04-19 07:01:44,1974-04-22 13:44:22,libxres-dev,/usr/bin/opam,<RECENT-CTIME>
1,1974-04-17 23:49:42,1974-04-14 13:33:16,libdesktop-agnostic0,/usr/bin/onboard,<OLD>
2,1974-04-23 02:58:48,1974-04-15 08:33:01,libdesktop-agnostic0,/usr/bin/opam,<OLD>
3,1974-04-18 21:06:40,1974-04-18 18:52:42,geoclue-ubuntu-geoip,/lib/init/upstart-job,nan
4,1974-04-14 00:56:26,1974-04-15 16:47:21,libqtdee-dev,/usr/bin/page,<RECENT-CTIME>


Now suppose we want to look at all packages that aren't libraries.

First, I want to get rid of everything with timestamp 0. Notice how we can just use a string in this comparison, even though it's actually a timestamp on the inside? That is because pandas is amazing.


In [27]:
popcon = popcon[popcon['atime'] > '1970-01-01']

Now we can use pandas' magical string abilities to just look at rows where the package name doesn't contain 'lib'.

In [28]:
nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]

In [29]:
nonlibraries.sort_values('ctime', ascending=False)[:10]

,atime,ctime,package-name,mru-program,tag
595,1974-04-21 05:15:07,1974-04-24 01:42:42,geoclue-ubuntu-geoip,/lib/init/upstart-job,<OLD>
2043,1974-04-20 12:31:31,1974-04-24 01:23:17,geoclue-ubuntu-geoip,/usr/bin/page,<OLD>
425,1974-04-18 13:47:35,1974-04-24 00:33:12,geoclue-ubuntu-geoip,/lib/init/upstart-job,<OLD>
2678,1974-04-23 16:56:46,1974-04-23 23:40:34,geoclue-ubuntu-geoip,/usr/bin/opam,<OLD>
2455,1974-04-13 17:55:38,1974-04-23 23:33:51,geoclue-ubuntu-geoip,/lib/init/upstart-job,<RECENT-CTIME>
1971,1974-04-13 16:02:47,1974-04-23 22:37:06,ubuntu-extras-keyring,/usr/bin/opam,<OLD>
30,1974-04-18 04:24:58,1974-04-23 22:26:33,ubuntu-extras-keyring,/lib/init/upstart-job,<OLD>
2544,1974-04-15 15:34:15,1974-04-23 22:17:50,ubuntu-extras-keyring,/usr/bin/onboard,<OLD>
497,1974-04-13 04:32:22,1974-04-23 22:08:39,ubuntu-extras-keyring,/lib/init/upstart-job,<RECENT-CTIME>
824,1974-04-18 12:51:57,1974-04-23 21:43:29,ubuntu-extras-keyring,/usr/bin/onboard,nan


Okay, cool, it says that I I installed ddd recently. And postgresql! I remember installing those things. Neat.

The whole message here is that if you have a timestamp in seconds or milliseconds or nanoseconds, then you can just "cast" it to a `'datetime64[the-right-thing]'` and pandas/numpy will take care of the rest.

## Putting it together

Now we want to request the full code execution.

Let's put all that together, to prove how easy it is. 6 lines of magical pandas!

In [30]:
@sy.syft_function(input_policy=sy.ExactMatch(popcon=ds.assets[0]),
                  output_policy=sy.SingleExecutionExactOutput())
def find_recently_installed(popcon):
    import pandas as pd
    popcon['atime'] = popcon['atime'].astype(int)
    popcon['ctime'] = popcon['ctime'].astype(int)
    popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
    popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')
    popcon = popcon[popcon['atime'] > '1970-01-01']
    nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]
    return nonlibraries.sort_values('ctime', ascending=False)[:10]

Request code execution

In [31]:
req = guest_domain_client.api.services.code.request_code_execution(find_recently_installed)

In [32]:
submitted_code = guest_domain_client.code[0]

In [33]:
assert guest_domain_client.api.services.code.get_all()

Create and submit project

In [34]:
new_project = sy.Project(name="Pandas Chapter 8",
                         description="Hi, I would like to get some insights about the installed programs")

In [35]:
new_project.add_request(obj=submitted_code, permission=sy.UserCodeStatus.EXECUTE)

In [36]:
guest_domain_client.submit_project(new_project)

<class 'syft.service.response.SyftSuccess'>: Project Submitted

# Data owner: execute syft_function

In [37]:
from syft import MessageStatus

In [38]:
domain_client = node.login(email="info@openmined.org", password="changethis")

# Get messages

In [39]:
messages = domain_client.api.services.messages.get_all_for_status(MessageStatus.UNDELIVERED)

In [40]:
messages

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,262be0b0aa6d4516a27e60998ac5eca6,Project Approval,MessageStatus.UNDELIVERED,2023-05-16 11:57:35,<<class 'syft.service.project.project.Project'...
1,syft.service.message.messages.Message,4eedad285e8a438b9f3fcba5d9343ec4,Approval Request,MessageStatus.UNDELIVERED,2023-05-16 11:57:34,<<class 'syft.service.request.request.Request'...


In [41]:
project_message = [x for x in messages if issubclass(x.linked_obj.object_type, Project)][0]

In [42]:
request = project_message.link.requests[0]
func = request.changes[0].link
op = func.output_policy_type

In [43]:
func

```python
class UserCode:
  id: str = d2ae442a487c48819abb6dc6ee26289e
  node_uid: str = eee436ccb7f044c4815c373f3fcf283d
  user_verify_key: str = 4280fb22b1e81d8516ab85bac61c5e1e856320fc3da356f5b1d5d1b10b707e08
  raw_code: str = "@sy.syft_function(input_policy=sy.ExactMatch(popcon=ds.assets[0]),
                  output_policy=sy.SingleExecutionExactOutput())
def find_recently_installed(popcon):
    import pandas as pd
    popcon['atime'] = popcon['atime'].astype(int)
    popcon['ctime'] = popcon['ctime'].astype(int)
    popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
    popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')
    popcon = popcon[popcon['atime'] > '1970-01-01']
    nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]
    return nonlibraries.sort_values('ctime', ascending=False)[:10]
"
  input_policy_type: str = <class 'syft.service.policy.policy.ExactMatch'>
  input_policy_init_kwargs: str = {NodeView(node_name='pandas-test-domain-1', verify_key=d5f240348a2aedc8216c38aae4e20f36c1cad31d587233e5c6b3347bd3e71747): {'popcon': <UID: dc3abbf673784c4da1dec3d75c776778>}}
  input_policy_state: str = b''
  output_policy_type: str = <class 'syft.service.policy.policy.OutputPolicyExecuteOnce'>
  output_policy_init_kwargs: str = {}
  output_policy_state: str = b''
  parsed_code: str = "def user_func_find_recently_installed_4280fb22b1e81d8516ab85bac61c5e1e856320fc3da356f5b1d5d1b10b707e08_76996a89952a0cd5f53e2e6da4ba9f2787211c4990d0fb1f69c5224b7af1b38c(popcon):

    def find_recently_installed(popcon):
        import pandas as pd
        popcon['atime'] = popcon['atime'].astype(int)
        popcon['ctime'] = popcon['ctime'].astype(int)
        popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
        popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')
        popcon = popcon[popcon['atime'] > '1970-01-01']
        nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]
        return nonlibraries.sort_values('ctime', ascending=False)[:10]
    result = find_recently_installed(popcon=popcon)
    return result"
  service_func_name: str = "find_recently_installed"
  unique_func_name: str = "user_func_find_recently_installed_4280fb22b1e81d8516ab85bac61c5e1e856320fc3da356f5b1d5d1b10b707e08_76996a89952a0cd5f53e2e6da4ba9f2787211c4990d0fb1f69c5224b7af1b38c"
  user_unique_func_name: str = "user_func_find_recently_installed_4280fb22b1e81d8516ab85bac61c5e1e856320fc3da356f5b1d5d1b10b707e08"
  code_hash: str = "76996a89952a0cd5f53e2e6da4ba9f2787211c4990d0fb1f69c5224b7af1b38c"
  signature: str = (popcon)
  status: str = {NodeView(node_name='pandas-test-domain-1', verify_key=d5f240348a2aedc8216c38aae4e20f36c1cad31d587233e5c6b3347bd3e71747): <UserCodeStatus.SUBMITTED: 'submitted'>}
  input_kwargs: str = ['popcon']
  enclave_metadata: str = None

```

In [44]:
find_recently_installed = func.unsafe_function

In [45]:
real_data = domain_client.datasets[0].assets[0].data.syft_action_data

In [46]:
real_data

,atime,ctime,package-name,mru-program,tag
0,1387295797,1367633260,perl-base,/usr/bin/perl,None
1,1387295796,1354370480,login,/bin/su,None
2,1387295743,1354341275,libtalloc2,/usr/lib/x86_64-linux-gnu/libtalloc.so.2.0.7,None
3,1387295743,1387224204,libwbclient0,/usr/lib/x86_64-linux-gnu/libwbclient.so.0,<RECENT-CTIME>
4,1387295742,1354341253,libselinux1,/lib/x86_64-linux-gnu/libselinux.so.1,None
...,...,...,...,...,...
2892,0,0,libreadline-dev,<NOFILES>,None
2893,0,0,notify-osd-icons,<NOFILES>,None
2894,0,0,python-apt-common,<NOFILES>,None
2895,0,0,libindicator-messages-status-provider1,<NOFILES>,None


In [47]:
real_result = find_recently_installed(popcon=real_data)

In [48]:
result = request.accept_by_depositing_result(real_result)
result
assert isinstance(result, sy.SyftSuccess)

# Data Owner: fetch result

In [49]:
guest_client.api.services.code[0].status

{NodeView(node_name='pandas-test-domain-1', verify_key=d5f240348a2aedc8216c38aae4e20f36c1cad31d587233e5c6b3347bd3e71747): <UserCodeStatus.EXECUTE: 'execute'>}

In [50]:
asset = guest_client.datasets[0].assets[0]

In [51]:
real_result = guest_client.api.services.code.find_recently_installed(popcon=asset)

In [52]:
real_result

,atime,ctime,package-name,mru-program,tag
57,2013-12-17 04:55:39,2013-12-17 04:55:42,ddd,/usr/bin/ddd,<RECENT-CTIME>
450,2013-12-16 20:03:20,2013-12-16 20:05:13,nodejs,/usr/bin/npm,<RECENT-CTIME>
454,2013-12-16 20:03:20,2013-12-16 20:05:04,switchboard-plug-keyboard,/usr/lib/plugs/pantheon/keyboard/options.txt,<RECENT-CTIME>
445,2013-12-16 20:03:20,2013-12-16 20:05:04,thunderbird-locale-en,/usr/lib/thunderbird-addons/extensions/langpac...,<RECENT-CTIME>
396,2013-12-16 20:08:27,2013-12-16 20:05:03,software-center,/usr/sbin/update-software-center,<RECENT-CTIME>
449,2013-12-16 20:03:20,2013-12-16 20:05:00,samba-common-bin,/usr/bin/net.samba3,<RECENT-CTIME>
397,2013-12-16 20:08:25,2013-12-16 20:04:59,postgresql-client-9.1,/usr/lib/postgresql/9.1/bin/psql,<RECENT-CTIME>
398,2013-12-16 20:08:23,2013-12-16 20:04:58,postgresql-9.1,/usr/lib/postgresql/9.1/bin/postmaster,<RECENT-CTIME>
452,2013-12-16 20:03:20,2013-12-16 20:04:55,php5-dev,/usr/include/php5/main/snprintf.h,<RECENT-CTIME>
440,2013-12-16 20:03:20,2013-12-16 20:04:54,php-pear,/usr/share/php/XML/Util.php,<RECENT-CTIME>


In [53]:
node.land()

Stopping pandas-test-domain-1
